[View in Colaboratory](https://colab.research.google.com/github/petroniocandido/PWFTS/blob/master/Google%20Colab%20-%20Computational%20Experiments.ipynb)

# Probabilistic Weighted FTS

Silva, P. C. L; Guimarães, F. G.; Sadaei, H. J. Probabilistic Forecasting With Fuzzy Time Series. Under publishing.


## Environment Setup

### Library install/update

In [1]:
!pip3 install raceplotly
!pip3 install -U git+https://opt/conda/lib/python3.7/site-packages (0.1.7)
!pip3 install -U git+https://opt/conda/lib/python3.7/site-packages (from raceplotly) (1.21.6)
!pip3 install -U git+https://opt/conda/lib/python3.7/site-packages (from raceplotly) (5.13.0)
!pip3 install -U git+https://opt/conda/lib/python3.7/site-packages (from raceplotly) (1.3.5)
!pip3 install -U git+https://opt/conda/lib/python3.7/site-packages (from pandas->raceplotly) (2.8.2)
!pip3 install -U git+https://opt/conda/lib/python3.7/site-packages (from pandas->raceplotly) (2022.7.1)
!pip3 install -U git+https://opt/conda/lib/python3.7/site-packages (from plotly->raceplotly) (8.1.0)
!pip3 install -U git+https://opt/conda/lib/python3.7/site-packages (from python-dateutil>=2.7.3->pandas->raceplotly) (1.16.0)
!pip install statsmodels

import statsmodels.api as sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `pip3 install -U git+https://opt/conda/lib/python3.7/site-packages (0.1.7)'
/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `pip3 install -U git+https://opt/conda/lib/python3.7/site-packages (from raceplotly) (1.21.6)'
/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `pip3 install -U git+https://opt/conda/lib/python3.7/site-packages (from raceplotly) (5.13.0)'
/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `pip3 install -U git+https://opt/conda/lib/python3.7/site-packages (from raceplotly) (1.3.5)'
/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `pip3 install -U git+https://opt/conda/lib/python3.7/site-packages (from pandas->raceplotly) (2.8.2)'
/bin/bash: -c:

In [2]:

!pip3 install -U git+https://github.com/petroniocandido/pyFTS
!pip3 install -U git+https://github.com/nicolaskruchten/jupyter_pivottablejs.git
!git clone https://github.com/petroniocandido/stac
!pip3 install dill
!wget https://github.com/petroniocandido/PWFTS/raw/master/benchmarks.db.gz
!gunzip benchmarks.db.gz
import pandas as pd


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/petroniocandido/pyFTS to /tmp/pip-req-build-sykjgzh6
  Running command git clone --filter=blob:none --quiet https://github.com/petroniocandido/pyFTS /tmp/pip-req-build-sykjgzh6
  Resolved https://github.com/petroniocandido/pyFTS to commit 4c77ecc73fa0888132ac3996ca7e1f519dcf2fc8
  Preparing metadata (setup.py) ... done
  Created wheel for pyFTS: filename=pyFTS-1.7-py3-none-any.whl size=217892 sha256=01db1fb2b33e0ea5461d19a49e2de8467a602b6daec2c32d5891ca6894b2f772
  Stored in directory: /tmp/pip-ephem-wheel-cache-wchua9km/wheels/10/44/09/24d848acb6e24a0d5aaddf1b5706e3691af4ce0293e0a2da29
Successfully built pyFTS
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/nicolaskruchten/jupyter_pivottablejs.git to /tmp/pip-req-build-8d864o2k
  Running command git clone --filter=blob:none --quie

### External libraries import

In [3]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf


In [ ]:
import matplotlib.pylab as plt

%pylab inline
import seaborn as sns

Populating the interactive namespace from numpy and matplotlib


In [4]:
import plotly.express as px

from raceplotly.plots import barplot
import plotly.offline as py
py.init_notebook_mode(connected=True)

### Common pyFTS imports and defaults

In [5]:
from pyFTS.common import Util as cUtil
from pyFTS.benchmarks import benchmarks as bchmk, Util as bUtil

from pyFTS.models import pwfts

from pyFTS.common import Transformations
tdiff = Transformations.Differential(1)

## Datasets

### Data Loading

In [6]:
df_f = pd.read_csv('Ferrari .csv')
df_t =  pd.read_csv('Tesla.csv')



FileNotFoundError: ignored

### Statistics

In [ ]:
df_f.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2015-10-22,57.070000,58.200001,55.700001,56.750000,53.604126,4545100
1,2015-10-23,57.770000,58.000000,56.270000,56.380001,53.254635,1967600
2,2015-10-26,57.000000,57.000000,54.540001,55.020000,51.970028,1466300
3,2015-10-27,54.799999,54.990002,49.360001,53.849998,50.864880,5949200
4,2015-10-28,54.020000,54.160000,50.099998,51.869999,48.994644,2411300


In [ ]:
new_df = pd.concat([df_f, df_t], axis=0)
new_df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2015-10-22,57.070000,58.200001,55.700001,56.750000,53.604126,4545100
1,2015-10-23,57.770000,58.000000,56.270000,56.380001,53.254635,1967600
2,2015-10-26,57.000000,57.000000,54.540001,55.020000,51.970028,1466300
3,2015-10-27,54.799999,54.990002,49.360001,53.849998,50.864880,5949200
4,2015-10-28,54.020000,54.160000,50.099998,51.869999,48.994644,2411300
...,...,...,...,...,...,...,...
1880,2023-04-13,182.960007,186.500000,180.940002,185.899994,185.899994,112933000
1881,2023-04-14,183.949997,186.279999,182.009995,185.000000,185.000000,96306500
1882,2023-04-17,186.320007,189.690002,182.690002,187.039993,187.039993,116662200
1883,2023-04-18,187.149994,187.690002,183.580002,184.309998,184.309998,92067000


In [ ]:
df_f.shape

In [ ]:
df_f['Name'] = 'Ferrari'
df_t.head()
df_t.shape
df_t['Name'] = 'Tesla'
df_f.dtypes

### Plots

In [ ]:
new_df["Date"] = pd.to_datetime(new_df["Date"],format="%Y-%m-%d")
new_df["Y"] = new_df["Date"].dt.year
new_df["M"] = new_df["Date"].dt.month
new_df["D"] = new_df["Date"].dt.day

In [ ]:
fig = px.line(new_df, x="Date", 
              y="High", color='Name',color_discrete_sequence = ['red', 'blue'],
              hover_data=['Low', 'Close','Adj Close','Volume'])


fig.show()



## Partitioning

Modificaçoes

In [ ]:


# Ajustar modelo SARIMA para Ferrari
model_ferrari = sm.tsa.SARIMAX(train['y_ferrari'], order=(1,1,1), seasonal_order=(0,1,1,12))
results_ferrari = model_ferrari.fit()

# Prever valores futuros para Ferrari
forecast_ferrari = results_ferrari.predict(start=len(train), end=len(train)+11, dynamic=False)

# Ajustar modelo SARIMA para Tesla
model_tesla = sm.tsa.SARIMAX(train['y_tesla'], order=(1,1,1), seasonal_order=(0,1,1,12))
results_tesla = model_tesla.fit()

# Prever valores futuros para Tesla
forecast_tesla = results_tesla.predict(start=len(train), end=len(train)+11, dynamic=False)

# Juntar as previsões em um único DataFrame
forecast = pd.DataFrame({'Date': test['ds'].values, 'Ferrari': forecast_ferrari.values, 'Tesla': forecast_tesla.values})

# Plotar as previsões
fig = px.line(new_df, x='ds', y='y_ferrari', title='Ferrari - Treino e Teste')
fig.add_trace(px.line(test, x='ds', y='y_ferrari').data[0])
fig.add_trace(px.line(forecast, x='Date', y='Ferrari').data[0])
fig.show()

fig = px.line(new_df, x='ds', y='y_tesla', title='Tesla - Treino e Teste')
fig.add_trace(px.line(test, x='ds', y='y_tesla').data[0])
fig.add_trace(px.line(forecast, x='Date', y='Tesla').data[0])
fig.show()


In [ ]:
from pyFTS.partitioners import Grid, Util as pUtil


### Partitioning exploring and optimization

**CAUTION**: This task is computationally expensive and take several hours to be performed. We strongly recommend to use the distributed version with a dispy cluster.

In [ ]:
from pyFTS.common import Transformations
from pyFTS.data import Tesla, Ferrari
from pyFTS.benchmarks import benchmarks as bchmk, Util as bUtil

tdiff = Transformations.Differential(1)

_type = 'point'
tag = "partitioning"

for dataset_name in dataset_names:
   dataset = [df_f, df_t]
   dataset_names = ['Ferrari','Tesla']

    bchmk.sliding_window_benchmarks(dataset, 1000, train=0.8, inc=0.2,
                                    methods=[pwfts.ProbabilisticWeightedFTS],
                                    benchmark_models=False,
                                    transformations=[None],
                                    #orders=[1,2,3],
                                    orders = [1],
                                    #partitions=np.arange(10,100,2), 
                                    partitions=np.arange(10,12), 
                                    progress=False, type=_type,
                                    steps_ahead=[1],
                                    #distributed=True, nodes=['192.168.0.110', '192.168.0.107','192.168.0.106'],
                                    file="benchmarks.db", dataset=dataset_name, tag=tag)
    
    """

    bchmk.sliding_window_benchmarks(dataset, 1000, train=0.8, inc=0.2,
                                    methods=[pwfts.ProbabilisticWeightedFTS],
                                    benchmark_models=False,
                                    transformations=[tdiff],
                                    orders=[1,2,3],
                                    partitions=np.arange(3,30,1), 
                                    progress=False, type=_type,
                                    steps_ahead=[1],
                                    #distributed=True, nodes=['192.168.0.110', '192.168.0.107', '192.168.0.106'],
                                    file="benchmarks.db", dataset=dataset_name, tag=tag)

    """

In [ ]:
df_f = bUtil.get_dataframe_from_bd("benchmarks.db",
                                  "tag = 'partitioning' and measure = 'rmse' and benchmarks.'order' = 1 " +
                                  "and transformation is null")

df_t = bUtil.get_dataframe_from_bd("benchmarks.db",
                                  "tag = 'partitioning' and measure = 'rmse' and benchmarks.'order' = 1 " +
                                  "and transformation is not null")

fig, ax = plt.subplots(nrows=2, ncols=1, figsize=[15,7])

g1 = sns.boxplot(x='Adj Close', y='Volume', hue='Data', data=df_f, showfliers=False, ax=ax[0], 
                 palette="Set3")
box = g1.get_position()
g1.set_position([box.x0, box.y0, box.width * 0.85, box.height]) 
g1.legend(loc='right', bbox_to_anchor=(1.15, 0.5), ncol=1)
ax[0].set_title("Original data")
ax[0].set_ylabel("RMSE")
ax[0].set_xlabel("")

g2 = sns.boxplot(x='Adj Close', y='Volume', hue='Data', data=df2, showfliers=False, ax=ax[1], 
                 palette="Set3")
box = g2.get_position()
g2.set_position([box.x0, box.y0, box.width * 0.85, box.height]) 
g2.legend(loc='right', bbox_to_anchor=(1.15, 0.5), ncol=1)
ax[1].set_title("Differentiated data")
ax[1].set_ylabel("RMSE")
ax[1].set_xlabel("Number of partitions of the UoD")


In [7]:
dat = bUtil.get_dataframe_from_bd("benchmarks.db","tag = 'partitioning' " 
                                  +" and dataset = 'TESLA' and transformation is not null")

from pyFTS.benchmarks import Util as bUtil
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()


df = dat[(dat.Measure == 'rmse') & (dat.Value <= 300)]

df['Value'] = min_max_scaler.fit_transform(df['Value'].reshape(-1, 1))

fig = plt.figure(figsize=[12,7])
ax = fig.gca(projection='3d')
ax.plot_trisurf(df['Order'], df['Partitions'], df['Value'], cmap="summer", linewidth=0.2, antialiased=True)
ax.set_title("Error surface by order and number of partitions")
ax.set_xlabel("Order")
ax.set_xticks([1,2,3])
ax.set_ylabel("Partitions")
ax.set_zlabel("RMSE")
ax.set_zlim([0,1])
plt.show()


AttributeError: ignored

### Default partitioners

In [ ]:
partitioner = Grid.GridPartitioner(data=dataset[:train_split], npart=50)
partitioner_diff = Grid.GridPartitioner(data=dataset[:train_split], npart=5, transformation=tdiff)

### Plots

In [ ]:

pUtil.plot_partitioners(dataset[:train_split], [partitioner,partitioner_diff], tam=[8,5])

## Model Training

### Without transformations 

In [ ]:
from pyFTS.common import fts,tree
from pyFTS.models import hofts, pwfts

pfts1_taiex = pwfts.ProbabilisticWeightedFTS(partitioner=partitioner)
pfts1_taiex.fit(dataset[:train_split], save_model=True, file_path='pwfts', order=1)
pfts1_taiex.shortname = "1st Order"
print(pfts1_taiex)

In [ ]:
pwfts.visualize_distributions(pfts1_taiex, size=(5,3))

### With transformations 

In [ ]:
from pyFTS.models import pwfts

pfts1_tesla_diff = pwfts.ProbabilisticWeightedFTS(partitioner=partitioner_diff)
pfts1_tesla_diff.append_transformation(tdiff)
pfts1_tesla_diff.fit(dataset[:train_split], order=2, save_model=True, file_path='pwfts_diff')
pfts1_tesla_diff.shortname = "Diff"
print(pfts1_tesla_diff)

In [ ]:
pwfts.visualize_distributions(pfts1_taiex_diff, size=(5,3))

## Test Forecasting

### Point Forecasting Benchmarks